In [1]:
from __future__ import print_function
import torch

# What is AUTOGRAD(AUTOMATIC DIFFERENTIATION)?

Central to all neural networks in PyTorch is the autograd package. Let’s first briefly visit this, and we will then go to training our first neural network.  
The ```autograd``` package provides automatic differentiation for all operations on Tensors. It is a define-by-run framework, which means that your backprop is defined by how your code is run, and that every single iteration can be different.  
Let us see this in more simple terms with some examples.  

## Tensor
```torch.Tensor``` is the central class of the package. If you set its attribute .requires_grad as ```True```, it starts to track all operations on it. When you finish your computation you can call ```.backward()``` and have all the gradients computed automatically. The gradient for this tensor will be accumulated into ```.grad``` attribute.

To stop a tensor from tracking history, you can call ```.detach()``` to detach it from the computation history, and to prevent future computation from being tracked.

To prevent tracking history (and using memory), you can also wrap the code block in ```with torch.no_grad():```. This can be particularly helpful when evaluating a model because the model may have trainable parameters with ```requires_grad=True```, but for which we don’t need the gradients.

There’s one more class which is very important for autograd implementation - a ```Function```.

```Tensor``` and ```Function``` are interconnected and build up an acyclic graph, that encodes a complete history of computation. Each tensor has a ```.grad_fn``` attribute that references a ```Function``` that has created the ```Tensor``` (except for Tensors created by the user - their ```grad_fn is None```).

If you want to compute the derivatives, you can call ```.backward()``` on a ```Tensor```. If ```Tensor``` is a scalar (i.e. it holds a one element data), you don’t need to specify any arguments to ```backward()```, however if it has more elements, you need to specify a ```gradient``` argument that is a tensor of matching shape.

In [2]:
x = torch.ones(2, 2, requires_grad=True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [3]:
y = x + 2
print(y)

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


In [4]:
print(y.grad_fn)

In [5]:
z = y * y * 3
out = z.mean()

print(z, out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


```.requires_grad_( ... )``` changes an existing Tensor’s requires_grad flag in-place. The input flag defaults to ```False``` if not given.

In [6]:

a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)

a.requires_grad_(True)
print(a.requires_grad)

b = (a * a).sum()
print(b.grad_fn)

False
True


## Gradients
Let’s backprop now. Because ```out``` contains a single scalar, ```out.backward()``` is equivalent to ```out.backward(torch.tensor(1.))```.

In [7]:
out.backward()

Print gradients d(out)/dx

In [8]:
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


Generally speaking, ```torch.autograd``` is an engine for computing vector-Jacobian product.  
Now let’s take a look at an example of vector-Jacobian product:

In [9]:
x = torch.randn(3, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:
    y = y * 2

print(y)

tensor([-1466.0022,   365.1219,    32.1075], grad_fn=<MulBackward0>)


Now in this case ```y``` is no longer a scalar. ```torch.autograd``` could not compute the full Jacobian directly, but if we just want the vector-Jacobian product, simply pass the vector to ```backward``` as argumenet:

In [10]:
v = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(v)

print(x.grad)

tensor([5.1200e+01, 5.1200e+02, 5.1200e-02])


You can also stop autograd from tracking history on Tensors with ```.requires_grad=True``` either by wrapping the code block in ```with torch.no_grad():```

In [11]:
print(x.requires_grad)
print((x ** 2).requires_grad)

with torch.no_grad():
    print((x ** 2).requires_grad)

True
True
False


Or by using ```.detach()``` to get a new Tensor with the same content but that does not require gradients:

In [12]:
print(x.requires_grad)
y = x.detach()
print(y.requires_grad)
print(x.eq(y).all())

True
False
tensor(True)


# Mnist with Logistic regression

In [13]:

def load_mnist_data():
    ## MNIST data
    from pathlib import Path
    import requests

    DATA_PATH = Path("../../data")
    PATH = DATA_PATH / "mnist"

    PATH.mkdir(parents=True, exist_ok=True)

    URL = "http://deeplearning.net/data/mnist/"
    FILENAME = "mnist.pkl.gz"

    if not (PATH / FILENAME).exists():
            content = requests.get(URL + FILENAME).content
            (PATH / FILENAME).open("wb").write(content)

    import pickle
    import gzip

    with gzip.open((PATH / FILENAME).as_posix(), "rb") as f:
            ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding="latin-1")
    return x_train, y_train, x_valid, y_valid

x_train, y_train, x_val, y_val = load_mnist_data()
print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)

(50000, 784) (50000,) (10000, 784) (10000,)


In [14]:
from matplotlib import pyplot as plt
import numpy as np

print(x_train.shape)

plt.imshow(x_train[0].reshape((28, 28)), cmap="gray")
plt.show()

(50000, 784)


In [15]:
import torch

x_train, y_train, x_val, y_val = map(
    torch.tensor, (x_train, y_train, x_val, y_val)
)

## Chack dimension
n, c = x_train.shape
print(x_train.shape)

## Check data
print(x_train)
print(y_train)
print(y_train.min(), y_train.max())
print(x_val)
print(y_val)
print(y_val.min(), y_val.max())

torch.Size([50000, 784])
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([5, 0, 4,  ..., 8, 4, 8])
tensor(0) tensor(9)
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
tensor([3, 8, 6,  ..., 5, 6, 8])
tensor(0) tensor(9)


In [16]:
import math

weights = torch.randn(784, 10) / math.sqrt(784)
weights.requires_grad_()
bias = torch.zeros(10, requires_grad=True)

In [57]:
def log_softmax(x):
    return x - x.exp().sum(-1).log().unsqueeze(-1)

def model(xb):
    return log_softmax(xb @ weights + bias)

In [58]:
def nll(pred, gt):
    return -pred[range(gt.shape[0]), gt].mean()

loss_func = nll

In [59]:
bs = 64  # batch size

xb = x_train[0:bs]  # a mini-batch from x

preds = model(xb)  # predictions
preds[0], preds.shape
print(preds[0], preds.shape)

yb = y_train[0:bs]
print(loss_func(preds, yb))

tensor([ -7.5445, -14.4752,  -8.8884,  -2.1302, -15.4838,  -0.1283, -11.9697,
         -7.2416,  -8.9616,  -9.8532], grad_fn=<SelectBackward>) torch.Size([64, 10])
tensor(0.1878, grad_fn=<NegBackward>)


In [60]:
xv = x_val[0:bs]

preds = model(xv)  # predictions
preds[0], preds.shape
print(preds[0], preds.shape)

yv = y_val[0:bs]
print(loss_func(preds, yv))


tensor([ -9.6079,  -4.0920,  -3.0031,  -0.1565, -12.3847,  -3.7476,  -8.3362,
        -17.4122,  -2.9064, -13.3414], grad_fn=<SelectBackward>) torch.Size([64, 10])
tensor(0.2577, grad_fn=<NegBackward>)


In [62]:
print(yb)

tensor([5, 0, 4, 1, 9, 2, 1, 3, 1, 4, 3, 5, 3, 6, 1, 7, 2, 8, 6, 9, 4, 0, 9, 1,
        1, 2, 4, 3, 2, 7, 3, 8, 6, 9, 0, 5, 6, 0, 7, 6, 1, 8, 7, 9, 3, 9, 8, 5,
        9, 3, 3, 0, 7, 4, 9, 8, 0, 9, 4, 1, 4, 4, 6, 0])


In [63]:
xb.shape, xb.dtype

(torch.Size([64, 784]), torch.float32)

In [64]:
weights.shape, weights.dtype

(torch.Size([784, 10]), torch.float32)

In [65]:
def accuracy(out, y):
    preds = torch.argmax(out, dim=1)
    return (preds == y).float().mean()

In [66]:
print(accuracy(preds, yb))

tensor(0.0938)


In [67]:
from IPython.core.debugger import set_trace

lr = 0.5  # learning rate
epochs = 5  # how many epochs to train for

for epoch in range(epochs):
    for i in range((n - 1) // bs + 1):
        
        start_i = i * bs
        end_i = start_i + bs
        xb = x_train[start_i:end_i]
        yb = y_train[start_i:end_i]
        
        ## Feed foward
        pred = model(xb)
        loss = loss_func(pred, yb)
        
        ## Backpropagation
        loss.backward()
        
        with torch.no_grad():
            weights -= weights.grad * lr
            bias -= bias.grad * lr
            weights.grad.zero_()
            bias.grad.zero_()
        
    print(loss_func(model(xb), yb), accuracy(model(xb), yb))
    print(loss_func(model(xv), yv), accuracy(model(xv), yv))

tensor(0.0594, grad_fn=<NegBackward>) tensor(1.)
tensor(0.2573, grad_fn=<NegBackward>) tensor(0.9375)
tensor(0.0579, grad_fn=<NegBackward>) tensor(1.)
tensor(0.2573, grad_fn=<NegBackward>) tensor(0.9375)
tensor(0.0567, grad_fn=<NegBackward>) tensor(1.)
tensor(0.2576, grad_fn=<NegBackward>) tensor(0.9375)
tensor(0.0558, grad_fn=<NegBackward>) tensor(1.)
tensor(0.2580, grad_fn=<NegBackward>) tensor(0.9375)
tensor(0.0550, grad_fn=<NegBackward>) tensor(1.)
tensor(0.2585, grad_fn=<NegBackward>) tensor(0.9375)


In [68]:
import torch.nn.functional as F

loss_func = F.cross_entropy

def model(xb):
    return xb @ weights + bias



In [69]:
print(loss_func(model(xb), yb), accuracy(model(xb), yb))
print(loss_func(model(xv), yv), accuracy(model(xv), yv))

tensor(0.0550, grad_fn=<NllLossBackward>) tensor(1.)
tensor(0.2585, grad_fn=<NllLossBackward>) tensor(0.9375)


In [70]:
from torch import nn

class Mnist_Logistic(nn.Module):
    def __init__(self):
        super().__init__()
        self.weights = nn.Parameter(torch.randn(784, 10) / math.sqrt(784))
        self.bias = nn.Parameter(torch.zeros(10))

    def forward(self, xb):
        return xb @ self.weights + self.bias

model = Mnist_Logistic()

In [71]:
print(loss_func(model(xb), yb))

tensor(2.3981, grad_fn=<NllLossBackward>)


In [72]:
with torch.no_grad():
    weights -= weights.grad * lr
    bias -= bias.grad * lr
    weights.grad.zero_()
    bias.grad.zero_()

In [73]:
loss = loss_func(model(xb), yb)
loss.backward()

with torch.no_grad():
    for p in model.parameters(): 
        print(p)
        p -= p.grad * lr
    model.zero_grad()

Parameter containing:
tensor([[ 0.0452, -0.0428,  0.0373,  ..., -0.0184,  0.0176,  0.0364],
        [-0.0242, -0.0061, -0.0417,  ..., -0.0438, -0.0625, -0.0186],
        [ 0.0792, -0.0260,  0.0107,  ..., -0.0110, -0.0066,  0.0119],
        ...,
        [ 0.0415,  0.0194,  0.0233,  ...,  0.0330, -0.0336, -0.0098],
        [-0.0070,  0.0200, -0.0268,  ..., -0.0305, -0.0103,  0.0404],
        [-0.0533,  0.0430, -0.0010,  ..., -0.0311,  0.0065,  0.0096]],
       requires_grad=True)
Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True)


In [74]:
def fit():
    for epoch in range(epochs):
        for i in range((n - 1) // bs + 1):
            start_i = i * bs
            end_i = start_i + bs
            xb = x_train[start_i:end_i]
            yb = y_train[start_i:end_i]
            pred = model(xb)
            loss = loss_func(pred, yb)
            loss.backward()
            
            with torch.no_grad():
                for p in model.parameters():
                    p -= p.grad * lr
                model.zero_grad()
        print("Epochs:", epoch, loss_func(model(xb), yb), accuracy(model(xb), yb), loss_func(model(xv), yv), accuracy(model(xv), yv))

fit()

Epochs: 0 tensor(0.1041, grad_fn=<NllLossBackward>) tensor(1.) tensor(0.2884, grad_fn=<NllLossBackward>) tensor(0.9531)
Epochs: 1 tensor(0.0792, grad_fn=<NllLossBackward>) tensor(1.) tensor(0.2742, grad_fn=<NllLossBackward>) tensor(0.9531)
Epochs: 2 tensor(0.0695, grad_fn=<NllLossBackward>) tensor(1.) tensor(0.2680, grad_fn=<NllLossBackward>) tensor(0.9375)
Epochs: 3 tensor(0.0645, grad_fn=<NllLossBackward>) tensor(1.) tensor(0.2648, grad_fn=<NllLossBackward>) tensor(0.9375)
Epochs: 4 tensor(0.0613, grad_fn=<NllLossBackward>) tensor(1.) tensor(0.2631, grad_fn=<NllLossBackward>) tensor(0.9375)


In [75]:
print('Train:', loss_func(model(xb), yb), accuracy(model(xb), yb), 'Validation:', loss_func(model(xv), yv), accuracy(model(xv), yv))

Train: tensor(0.0613, grad_fn=<NllLossBackward>) tensor(1.) Validation: tensor(0.2631, grad_fn=<NllLossBackward>) tensor(0.9375)
